<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="cognitiveclass.ai logo"  />
</center>

# Investigation of cryptocurrency exchange rate dynamic (Matic/USD), сalculation and analysis of technical financial indicators, characterizing the cryptocurrency market (ATR, OBV, RSI, AD)

## Lab 3. Exploratoy Data Analysis

Estimated time needed: **30** minutes

## Objectives

After completing this lab you will be able to:

*   Explore features or charecteristics to predict volume of Matic

### **Columns**

* #### `Ts` - the timestamp of the record
* #### `Open` -  the price of the asset at the beginning of the trading period
* #### `High` -  the highest price of the asset during the trading period
* #### `Low` - the lowest price of the asset during the trading period.
* #### `Close` - the price of the asset at the end of the trading period
* #### `Volume` - the total number of shares or contracts of a particular asset that are traded during a given period
* #### `Rec_count` -  the number of individual trades or transactions that have been executed during a given time period
* #### `Avg_price` - the average price at which a particular asset has been bought or sold during a given period
* #### `ATR` - average true range indicator
* #### `OBV` - on-balance volume indicator
* #### `RSI` - relative strength index indicator
* #### `AD` - accumulation / distribution indicator
* #### `BTC_price` - the avarage price from BTC/BUSD dataset 
* #### `BNB_price` - the avarage price from BNB/BUSD dataset
* ##### Additional columns:  'Open_EUR', 'BTC_price_EUR', 'high_EUR', 'High_Normalized', 'Low_Normalized','close_low', 'close_medium', 'close_high', 'rec_count-binned','rec_count_low', 'rec_count_medium', 'rec_count_high'

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
<ol>
    <li><a href="https://#import_data">Import Data from Module</a></li>
    <li><a href="https://#pattern_visualization">Analyzing Individual Feature Patterns using Visualization</a></li>
    <li><a href="https://#discriptive_statistics">Descriptive Statistical Analysis</a></li>
    <li><a href="https://#basic_grouping">Basics of Grouping</a></li>
    <li><a href="https://#correlation_causation">Correlation and Causation</a></li>
    <li><a href="https://#anova">ANOVA</a></li>
</ol>

</div>

<hr>


### What are the main characteristics that have the most impact on the Matic volume?


## 1. Import Data from Module 2


#### Setup


you are running the lab in your  browser, so we will install the libraries using `piplite`


In [ ]:
! pip install scikit-learn 
! pip install mplfinance

In [ ]:
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import numpy as np
import pandas as pd 
import numpy as np 
import mplfinance as fplt
from scipy import stats
from statsmodels.tsa.stattools import grangercausalitytests
import statsmodels.api as sm
from statsmodels.stats.stattools import durbin_watson as dwtest
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline 
#set precision 
pd.set_option("display.precision", 2)
#set precision for float
pd.options.display.float_format = '{:.2f}'.format

Import libraries:


If you run the lab locally using Anaconda, you can load the correct library and versions by uncommenting the following:


In [ ]:
#If you run the lab locally using Anaconda, you can load the correct library and versions by uncommenting the following:
#install specific version of libraries used in lab
#! mamba install pandas==1.3.3
#! mamba install numpy=1.21.2
#! mamba install scipy=1.7.1-y
#!  mamba install seaborn=0.9.0-y

This function will download the dataset into your browser


In [ ]:
##Now you need to use dataset, you made in he first lab 
filename = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-GPXX0YYYEN/Lab2DataSet.csv"
df = pd.read_csv(filename,low_memory=False, index_col=0)
df

Load the data and store it in dataframe `df`:


This dataset was hosted on IBM Cloud object. Click <a href="https://cocl.us/DA101EN_object_storage?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDA0101ENSkillsNetwork20235326-2021-01-01">HERE</a> for free storage.


## 2. Analyzing Individual Feature Patterns Using Visualization


To install Seaborn we use pip, the Python package manager.


#### How to choose the right visualization method?
<p>When visualizing individual variables, it is important to first understand what type of variable you are dealing with. This will help us find the right visualization method for that variable.</p>


In [ ]:
#set correct types
df.index = df.index.astype("datetime64[ns]")
df.head()

In [ ]:
# list the data types for each column
print(df.dtypes)

Before we start, let's look at the graph of Matic fluctuations. 

Make a new dataframe for our graph:

In [ ]:
# make a dataframe
box = pd.DataFrame()
# set columns
box['Open'] = df['Open']
box['High'] = df['High']
box['Low'] = df['Low']
box['Close'] = df['Close']
box.index = df.index
# drop all NaN
box = box.dropna()
# set column 'time' as index
box.head()


Use function <code>fplt.plot</code> for our graphic:

In [ ]:
fplt.plot(box.head(50),
          type='candle',
          style='charles',
          title='MATIC',
          ylabel='Price ($)')


<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h3>Question  #1:</h3>

<b>What is the data type of the column "Volume"? </b>

</div>


In [ ]:
# Write your code below and press Shift+Enter to execute 
df['Volume'].dtypes

<details><summary>Click here for the solution</summary>

```python
df['Volume'].dtypes
```

</details>


For example, we can calculate the correlation between all variables with different types using the method "corr":


In [ ]:
corr = df.corr()
corr

Here is correlation between Matic Volume,Rec_cout, also between BTC and BNB prices:


In [ ]:
corr = df[["Volume","Rec_count","BTC_price","BNB_price"]].corr()
sns.heatmap(corr)

The diagonal elements are always one; we will study correlation more precisely Pearson correlation in-depth at the end of the notebook.


<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h3> Question  #2: </h3>

<p>Find the correlation between the following columns: Volume, Close ; and built a heatmap for them.</p>
<p>Hint: if you would like to select those columns, use the following syntax:corr = df[["Open","Close"]]</p>
</div>


In [ ]:
corr = df[["Volume","Close"]].corr()
sns.heatmap(corr)

<details><summary>Click here for the solution</summary>

```python
corr = df[["Volume","Close"]].corr()
sns.heatmap(corr)
```

</details>


## Continuous Numerical Variables:

<p>Continuous numerical variables are variables that may contain any value within some range. They can be of type "int64" or "float64". A great way to visualize these variables is by using scatterplots with fitted lines.</p>

<p>In order to start understanding the (linear) relationship between a Volume and a Rec_count for Matic, we can use "regplot" which plots the scatterplot plus the fitted regression line for the data.</p>


Let's see several examples of different linear relationships:


<h3>Positive Linear Relationship</h4>


Let's find the scatterplot of "Rec_count" and "Volume".


In [ ]:
# volume price as potential predictor variable of volume
sns.regplot(x="Rec_count", y="Volume", data=df,scatter_kws={'s':2})
plt.ylim(200,)

<p>As the Rec_count goes up, the Volume goes up: this indicates a positive direct correlation between these two variables. Rec_cont seems like a good predictor of Volume since the regression line is almost a perfect diagonal line.</p>


We can examine the correlation between 'Volume' and 'Rec_count' and see that it's approximately 0.68.


In [ ]:
df[["Rec_count", "Volume"]].corr()

### Weak Linear Relationship


We have some financial indicators like "OBV", "RSI", "AD" and "ATR" in our dataset, lets check the correlation between Volume and indicators:

In [ ]:
df[["Volume", "OBV","RSI","AD","ATR"]].corr()

Also make a heatmap:

In [ ]:
sns.heatmap(df[["Volume", "OBV","RSI","AD","ATR"]].corr())

As we can see in the table and on the map, the strongest correlation between the volume and the indicators is ATR with a value of 0.55, which is not an indicator of a strong dependency.

Let's see if "BNB_price" is a predictor variable of "Volume".


In [ ]:
sns.regplot(x="BTC_price", y="Volume", data=df,scatter_kws={'s':2})

<p>BTC_price does not seem like a good predictor of the Volume at all since the regression line is close to horizontal. Also, the data points are very scattered and far from the fitted line, showing lots of variability. Therefore, it's not a reliable variable.</p>


We can examine the correlation between 'BTC_price' and 'Volume' and see it's approximately -0.20.


In [ ]:
df[['BTC_price','Volume']].corr()

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
    <b style="font-size: 2em; font-weight: bold;"> Question  3 a): </b><br>
    <p>Find the correlation  between "Volume" and "ATR".</p>
    <p>Hint: if you would like to select those columns, use the following syntax:df[["ATR","Volume"]].corr()  </p>
</div>


In [ ]:
# Write your code below and press Shift+Enter to execute
df[["ATR","Volume"]].corr()

<details><summary>Click here for the solution</summary>

```python

#The correlation is 0.491169.

df[["Volume","ATR"]].corr()

```

</details>


<div class="alert alert-danger alertdanger" style="margin-top: 20px">
    <b style="font-size: 2em; font-weight: bold;">Question  3 b):</b><br>
    <p>Given the correlation results between "
    ATR" and "Volume", do you expect a linear relationship?</p> 
    <p>Verify your results using the function "regplot()".</p>
</div>


In [ ]:
# Write your code below and press Shift+Enter to execute 
sns.regplot(x="ATR", y="Volume", data=df,scatter_kws={'s':2})

<details><summary>Click here for the solution</summary>

```python

sns.regplot(x="ATR", y="Volume", data=df,scatter_kws={'s':2})

```

</details>


### Make Categorical Variables for next point
<p>Let`s make categorical values as in the past lab.</p>

<p>We would like 5 bins of equal size bandwidth so we use numpy's <code>linspace(start_value, end_value, numbers_generated</code> function.</p>
<p>Since we want to include the minimum value of open, we want to set start_value = min(df["open"]).</p>
<p>Since we want to include the maximum value of open, we want to set end_value = max(df["open"]).</p>
<p>Since we are building 5 bins of equal length, there should be 6 dividers, so numbers_generated = 6.</p>


We build a bin array with a minimum value to a maximum value by using the bandwidth calculated above. The values will determine when one bin ends and another begins.


We set group  names:


In [ ]:
group_names = ['Low','Lower Medium' ,'Medium', 'Upper Medium' ,'High']

We have to make some fields categorical, lets make a function which make it

In [ ]:
def to_categorical(column: pd.Series, labels: list) -> pd.Series:
    cat_number = len(labels)
    bins = np.linspace(min(column), max(column), cat_number+1)
    #We apply the function "cut" to determine what each value of column belongs to.
    res = pd.cut(column, bins, labels=labels, include_lowest=True)
    return res

Now lets use our function with: Avg_price, BTC_price, BNB_price:

In [ ]:
df['Avg_price-binned'] = to_categorical(df["Avg_price"],group_names)
df[["Avg_price", "Avg_price-binned"]].head()

In [ ]:
df['BTC_price-binned'] = to_categorical(df["BTC_price"],group_names)
df[["BTC_price", "BTC_price-binned"]].head()

In [ ]:
df['BNB_price-binned'] = to_categorical(df["BNB_price"],group_names)
df[["BNB_price", "BNB_price-binned"]].head()

In [ ]:
df['Rec_count-binned'] = to_categorical(df["Rec_count"],group_names)
df[["Rec_count", "Rec_count-binned"]].head()

<p>These are variables that describe a 'characteristic' of a data unit, and are selected from a small group of categories. The categorical variables can have the type "object" or "int64". A good way to visualize categorical variables is by using boxplots.</p>


Let's look at the relationship between "Avg_price-binned" and "Volume".


In [ ]:
sns.boxplot(x="Avg_price-binned", y="Volume", data=df)

<p>Here we see that the distribution of price between these five categories are distinct enough to take in which catagery price will be as a potential good predictor of price. Let's examine engine "BTC_price-binned" and "Volume":</p>


In [ ]:
sns.boxplot(x="BTC_price-binned", y="Volume", data=df)


<p>We can see the same.</p>


Let's examine "BNB_price-binned" and "Volume".


In [ ]:
# drive-wheels
sns.boxplot(x="BNB_price-binned", y="Volume", data=df)

<p><p>We see that the distributions of price between the different categories have a significant overlap, so BNB_price-binned would not be a good predictor of Volume. </p>
</p>


<p>Let's examine "Rec_count" and "Volume".
</p>


In [ ]:
# drive-wheels
sns.boxplot(x="Rec_count-binned", y="Volume", data=df)

## 3. Descriptive Statistical Analysis


<p>Let's first take a look at the variables by utilizing a description method.</p>

<p>The <b>describe</b> function automatically computes basic statistics for all continuous variables. Any NaN values are automatically skipped in these statistics.</p>

This will show:

<ul>
    <li>the count of that variable</li>
    <li>the mean</li>
    <li>the standard deviation (std)</li> 
    <li>the minimum value</li>
    <li>the IQR (Interquartile Range: 25%, 50% and 75%)</li>
    <li>the maximum value</li>
<ul>


We can apply the method "describe" as follows:


In [ ]:
df.describe()

The default setting of "describe" skips variables of type category. We can apply the method "describe" on the variables of type 'category' as follows:


In [ ]:
df.describe(include='category')

### Value Counts


<p>Value counts is a good way of understanding how many units of each characteristic/variable we have. We can apply the "value_counts" method on the column "Volume". Don’t forget the method "value_counts" only works on pandas series, not pandas dataframes. As a result, we only include one bracket <code>df['Volume']</code>, not two brackets <code>df[['Volume']]</code>.</p>


In [ ]:
df['Volume'].value_counts()

We can convert the series to a dataframe as follows:


In [ ]:
df['Volume'].value_counts().to_frame()

Let's repeat the above steps but save the results to the dataframe "Avg_price_binned_counts" and rename the column  'Avg_price-binned' to 'value_counts'.


In [ ]:
Volume_counts = df['Volume'].value_counts().to_frame()
Volume_counts.rename(columns={'Volume': 'volume_counts'}, inplace=True)
Volume_counts

Now let's rename the index to 'Avg_price-binned':


In [ ]:
Volume_counts.index.name = 'Volume_counts'
Volume_counts

<p>After examining the value counts of the Avg_price-binned, we see that this category would be a good predictor variable for the price. This is because we only have mostly our prices in 'Medium'.</p>


## 4. Basics of Grouping


<p>The "groupby" method groups data by different categories. The data is grouped based on one or several variables, and analysis is performed on the individual groups.</p>

<p>For example, let's group by the variable "Avg_price-binned". We see that there are 5 different categories of Avg_price-binned.</p>


In [ ]:
df['Volume'].unique()

<p>If we want to know, on average, which type of Avg_price-binned is most valuable, we can group "Avg_price-binned" and then average them.</p>

<p>We can select the columns 'Avg_price-binned' and 'Avg_price', then assign it to the variable "df_group_one".</p>


In [ ]:
df_group_one = df[['Avg_price-binned','Volume']]

We can then calculate the average price for each of the different categories of data.


In [ ]:
# grouping results
df_group_one = df_group_one.groupby(['Avg_price-binned'],as_index=False).mean()
df_group_one

<p>From our data, it seems High category are the most expensive.</p>

<p>You can also group by multiple variables. For example, let's group by both 'Avg_price-binned' and 'BTC_price-binned'. This groups the dataframe by the unique combination of 'Avg_price-binned' and 'Avg_price-binned'. We can store the results in the variable 'grouped_test1'.</p>


In [ ]:
# grouping results
df_gptest = df[['BTC_price-binned','Volume']]
grouped_test1 = df_gptest.groupby(['BTC_price-binned'],as_index=False).mean()
grouped_test1

<p>This grouped data is much easier to visualize when it is made into a pivot table. A pivot table is like an Excel spreadsheet, with one variable along the column and another along the row. We can convert the dataframe to a pivot table using the method "pivot" to create a pivot table from the groups.</p>

<p>In this case, we will leave the Avg_price-binned variable as the rows of the table, and BTC_price-binned to become the columns of the table:</p>


In [ ]:
grouped_pivot = pd.crosstab(index=grouped_test1['BTC_price-binned'],columns=grouped_test1['Volume'])
grouped_pivot

<p>Often, we won't have data for some of the pivot cells. We can fill these missing cells with the value 0, but any other value could potentially be used as well. It should be mentioned that missing data is quite a complex subject and is an entire course on its own.</p>


In [ ]:
grouped_pivot = grouped_pivot.fillna(0) #fill missing values with 0
grouped_pivot

Also we can use a crossed table to see how many values correspond to each other in the table:

In [ ]:
crossed_table = pd.crosstab(df['Avg_price-binned'],df['Rec_count-binned'])
crossed_table

As we can see, mostly they correspond when bin for BTC is 'Medium' and bin for Avg_price is 'Medium'.

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
    <b style="font-size: 2em; font-weight: bold;">Question 4:</b><br>
    <p>Use the "groupby" function to find the average "BTC_price" of each trade based on "BTC_price-binned".</p>
</div>


In [ ]:
# Write your code below and press Shift+Enter to execute 
# grouping results
df_gptest2 = df[['BTC_price','Volume']]
grouped_test_bodystyle = df_gptest2.groupby(['BTC_price'],as_index= False).mean()
grouped_test_bodystyle

<details><summary>Click here for the solution</summary>

```python
df_gptest2 = df[['BTC_price','Volume']]
grouped_test_bodystyle = df_gptest2.groupby(['BTC_price'],as_index= False).mean()
grouped_test_bodystyle
```

</details>


#### Variables: BTC_price-binned vs. Avg_price-binned


Let's use a heat map to visualize the relationship between BTC_price-binned vs Avg_price-binned.

In [ ]:
#use the grouped results
plt.grid(False)
plt.pcolor(crossed_table, cmap='RdBu')
plt.show()

<p>The heatmap plots the target variable (price) proportional to colour with respect to the variables 'Avg_price-binned' and 'BTC_price-binned' on the vertical and horizontal axis, respectively. This allows us to visualize how the price is related to 'BTC_price-binned' and 'Avg_price-binned'.</p>

<p>The default labels convey no useful information to us. Let's change that:</p>


In [ ]:
fig, ax = plt.subplots()
plt.grid(False)

im = ax.pcolor(crossed_table, cmap='RdBu')

# Set the row and column labels
row_labels = crossed_table.columns
col_labels = crossed_table.index

# Set the x and y tick positions to the center of each cell
ax.set_xticks(np.arange(crossed_table.shape[1]) + 0.5)
ax.set_yticks(np.arange(crossed_table.shape[0]) + 0.5)

# Set the tick labels
ax.set_xticklabels(row_labels, rotation=90)
ax.set_yticklabels(col_labels)

plt.show()

<p>Visualization is very important in data science, and Python visualization packages provide great freedom. We will go more in-depth in a separate Python visualizations course.</p>

<p>The main question we want to answer in this module is, "What are the main characteristics that have the most impact on the MATIC price?".</p>

<p>To get a better measure of the important characteristics, we look at the correlation of these variables with the MATIC avg price. In other words: how is the MATIC avg price dependent on this variable?</p>


## 5. Correlation and Causation


<p><b>Correlation</b>: a measure of the extent of interdependence between variables.</p>

<p><b>Causation</b>: the relationship between cause and effect between two variables.</p>

<p>It is important to know the difference between these two. Correlation does not imply causation. Determining correlation is much simpler  the determining causation as causation may require independent experimentation.</p>


<p><b>Pearson Correlation</b></p>
<p>The Pearson Correlation measures the linear dependence between two variables X and Y.</p>
<p>The resulting coefficient is a value between -1 and 1 inclusive, where:</p>
<ul>
    <li><b>1</b>: Perfect positive linear correlation.</li>
    <li><b>0</b>: No linear correlation, the two variables most likely do not affect each other.</li>
    <li><b>-1</b>: Perfect negative linear correlation.</li>
</ul>


<p>Pearson Correlation is the default method of the function "corr". Like before, we can calculate the Pearson Correlation of the of the 'int64' or 'float64'  variables.</p>


In [ ]:
df.corr()

Sometimes we would like to know the significant of the correlation estimate.


<b>P-value</b>

<p>What is this P-value? The P-value is the probability value that the correlation between these two variables is statistically significant. Normally, we choose a significance level of 0.05, which means that we are 95% confident that the correlation between the variables is significant.</p>

By convention, when the

<ul>
    <li>p-value is $<$ 0.001: we say there is strong evidence that the correlation is significant.</li>
    <li>the p-value is $<$ 0.05: there is moderate evidence that the correlation is significant.</li>
    <li>the p-value is $<$ 0.1: there is weak evidence that the correlation is significant.</li>
    <li>the p-value is $>$ 0.1: there is no evidence that the correlation is significant.</li>
</ul>


<h3>"Volume" vs. "BNB_price"</h3>


Let's calculate the  Pearson Correlation Coefficient and P-value of "Avg_price" and "BNB_price".


In [ ]:
pearson_coef, p_value = stats.pearsonr(df["Volume"], df["BNB_price"])
print("The Pearson Correlation Coefficient is", round(pearson_coef, 2), " with a P-value of P =", round(p_value, 2))  

#### Conclusion:
<p>Since the p-value is $<$ 0.001, the correlation between "Avg_price" and "BNB_price" is statistically significant and the linear relationship is strong enough (≈ 0.63).</p>


### "Volume" vs. "BTC_price"


Let's calculate the  Pearson Correlation Coefficient and P-value of "Avg_price" and "BTC_price".


In [ ]:
pearson_coef, p_value = stats.pearsonr(df["Volume"], df["Rec_count"])
print("The Pearson Correlation Coefficient is", round(pearson_coef, 2), " with a P-value of P = ", p_value)  

<h4>Conclusion:</h4>

<p>Since the p-value is $<$ 0.001, the correlation between "ETHBUSD_avg_price" and "BNBBUSD_avg_price" is statistically significant, and the linear relationship is quite weak (≈ 0.37).</p>


<b style="font-size: 2em; font-weight: bold;"> Volume" vs. "BTC_price"</b>

Let's calculate the  Pearson Correlation Coefficient and P-value of "BNB_price" and "BTC_price".


In [ ]:
pearson_coef, p_value = stats.pearsonr(df["Volume"], df["ATR"])
print("The Pearson Correlation Coefficient is", round(pearson_coef, 2), " with a P-value of P = ", p_value)  

#### Conclusion:

Since the p-value is < 0.001, the correlation between "BNB_price" and "BTC_price" is statistically significant and the linear relationship is weak (≈ 0.06).


### "Volume" vs. "ATR"


Let's calculate the Pearson Correlation Coefficient and P-value of "BNBBUSD_avg_price" and "APEBUSD_avg_price":


In [ ]:
pearson_coef, p_value = stats.pearsonr(df["Volume"], df["ATR"])
print( "The Pearson Correlation Coefficient is", round(pearson_coef, 2), " with a P-value of P = ", p_value)  

#### Conclusion:
<p>Since the p-value is $<$ 0.001, the correlation between "Avg_price" and "ATR" is statistically significant, and the linear relationship is only moderate (≈ 0.31).</p>


### "Volume" vs. "OBV"


Let's calculate the  Pearson Correlation Coefficient and P-value of "Avg_price" and "OBV":


In [ ]:
pearson_coef, p_value = stats.pearsonr(df["Volume"], df["OBV"])
print("The Pearson Correlation Coefficient is", round(pearson_coef, 2), " with a P-value of P =  ", p_value ) 

#### Conclusion:
<p>Since the p-value is $<$ 0.001, the correlation between "Avg_price" and "OVB" is statistically significant and the linear relationship is only moderate(≈ 0.19).</p>


## 6. ANOVA


### ANOVA: Analysis of Variance
<p>The Analysis of Variance  (ANOVA) is a statistical method used to test whether there are significant differences between the means of two or more groups. ANOVA returns two parameters:</p>

<p><b>F-test score</b>: ANOVA assumes the means of all groups are the same, calculates how much the actual means deviate from the assumption, and reports it as the F-test score. A larger score means there is a larger difference between the means.</p>

<p><b>P-value</b>:  P-value tells how statistically significant our calculated score value is.</p>

<p>If our price variable is strongly correlated with the variable we are analyzing, we expect ANOVA to return a sizeable F-test score and a small p-value.</p>


### BNBBUSD_ap_cat


<p>Since ANOVA analyzes the difference between different groups of the same variable, the groupby function will come in handy. Because the ANOVA algorithm averages the data automatically, we do not need to take the average before hand.</p>

In [ ]:
grouped_test2 = df[["Avg_price-binned", "Volume"]].groupby(["Avg_price-binned"])
grouped_test2.head()

We can obtain the values of the method group using the method "get_group".


In [ ]:
grouped_test2.get_group("Medium")["Volume"]

In [ ]:
grouped_test2.head()

We can use the function "f_oneway" in the module "stats" to obtain the <b>F-test score</b> and <b>P-value</b>.


In [ ]:
# ANOVA
f_val, p_val = stats.f_oneway(grouped_test2.get_group("Low")["Volume"], grouped_test2.get_group("Lower Medium")["Volume"], grouped_test2.get_group("Medium")["Volume"], grouped_test2.get_group("Upper Medium")["Volume"], grouped_test2.get_group("High")["Volume"])  
 
print("ANOVA results: F=", round(f_val, 2), ", P =", round(p_val, 2))

This is a great result with a large F-test score showing a strong correlation and a P-value of 0 implying almost certain statistical significance. But does this mean all five tested groups are all this highly correlated?

Let's examine them separately.


#### Low and Lower Medium


In [ ]:
f_val, p_val = stats.f_oneway(grouped_test2.get_group("Low")["Volume"], grouped_test2.get_group("Lower Medium")["Volume"])  
 
print( "ANOVA results: F=", round(f_val, 2), ", P =", round(p_val, 2) )

Let's examine the other groups.


#### Low and Medium


In [ ]:
f_val, p_val = stats.f_oneway(grouped_test2.get_group("Low")["Volume"], grouped_test2.get_group("Medium")["Volume"])  
   
print( "ANOVA results: F=", round(f_val, 2), ", P =", round(p_val, 2))   

#### Low and Upper Medium


In [ ]:
f_val, p_val = stats.f_oneway(grouped_test2.get_group("Low")["Volume"], grouped_test2.get_group("Upper Medium")["Volume"])  
 
print("ANOVA results: F=", round(f_val, 2), ", P =", round(p_val, 2))   

<h4>Low and High</h4>


In [ ]:
f_val, p_val = stats.f_oneway(grouped_test2.get_group("Low")["Volume"], grouped_test2.get_group("High")["Volume"])  
 
print("ANOVA results: F=", round(f_val, 2), ", P =", p_val)   

#### Lower Medium and Medium


In [ ]:
f_val, p_val = stats.f_oneway(grouped_test2.get_group("Lower Medium")["Volume"], grouped_test2.get_group("Medium")["Volume"])  
 
print("ANOVA results: F=", round(f_val, 2), ", P =", round(p_val, 2))   

#### Lower Medium and Upper Medium


In [ ]:
f_val, p_val = stats.f_oneway(grouped_test2.get_group("Lower Medium")["Volume"], grouped_test2.get_group("Upper Medium")["Volume"])  
 
print("ANOVA results: F=", round(f_val, 2), ", P =", p_val)   

#### Lower Medium and High


In [ ]:
f_val, p_val = stats.f_oneway(grouped_test2.get_group("Lower Medium")["Volume"], grouped_test2.get_group("High")["Volume"])  
 
print("ANOVA results: F=", round(f_val, 2), ", P =", round(p_val, 2))   

#### Medium and Upper Medium


In [ ]:
f_val, p_val = stats.f_oneway(grouped_test2.get_group("Medium")["Volume"], grouped_test2.get_group("Upper Medium")["Volume"])  
 
print("ANOVA results: F=", round(f_val, 2), ", P =", round(p_val, 2))   

#### Medium and High


In [ ]:
f_val, p_val = stats.f_oneway(grouped_test2.get_group("Medium")["Volume"], grouped_test2.get_group("High")["Volume"])  
 
print("ANOVA results: F=", f_val, ", P =", round(p_val, 2))   

#### Upper Medium and High


In [ ]:
f_val, p_val = stats.f_oneway(grouped_test2.get_group("Upper Medium")["Volume"], grouped_test2.get_group("High")["Volume"])  
 
print("ANOVA results: F=", round(f_val, 2), ", P =", round(p_val, 2))   

## 7. Durbin-Watson Test

#### What is Durbin-Watson Test?

In regression analysis, Durbin-Watson (DW) is useful for checking the first-order autocorrelation (serial correlation). It analyzes the residuals for independence over time points (autocorrelation). The autocorrelation varies from -1 (negative autocorrelation) to 1 (positive autocorrelation).

Durbin-Watson test analyzes the following hypotheses,

Null hypothesis (H<sub>0</sub>): Residuals from the regression are not autocorrelated (autocorrelation coefficient, ρ = 0)
Alternative hypothesis (H<sub>a</sub>): Residuals from the regression are autocorrelated (autocorrelation coefficient, ρ > 0)

We will use <b>durbin_watson</b> for Durbin-Watson Test and <b>OLS</b> to get residuals from "statsmodels" library

In [ ]:
X = df["Avg_price"] # independent
y = df["Volume"] # dependent
# to get intercept
X = sm.add_constant(X)
# fit the regression model
reg = sm.OLS(y, X).fit()

In [ ]:
dwtest(resids=np.array(reg.resid))

In [ ]:

currs = ["Volume", "BTC_price", "BNB_price"]
import itertools

cols = [curr for curr in currs]
idxs = [curr for curr in currs]

dw_df = pd.DataFrame(columns=cols, index=idxs)

for (curr1, curr2) in itertools.permutations(currs, 2):
    X = df[{curr1}] # independent
    y = df[{curr2}] # dependent
    # to get intercept
    X = sm.add_constant(X)
    # fit the regression model
    reg = sm.OLS(y, X).fit()
    dw = dwtest(resids=np.array(reg.resid))
    dw_df.loc[curr2 ,curr1] = dw
    
np.fill_diagonal(dw_df.values, "—")
    
dw_df

Above we can see a matrix that consists of all the p-values. How to interpret these values? We take any value, its column will be responsible for the dependent value, and the row - for the independent value

## 8. Granger Causality Test

#### What is Granger Causality Test?

Granger Causality test is a statistical test that is used to determine if a given time series and it’s lags is helpful in explaining the value of another series. 

The Null hypothesis for grangercausalitytests is that the time series in
the second column, x<sub>2</sub>, does NOT Granger cause the time series in the first
column, x<sub>1</sub>. Grange causality means that past values of x<sub>2</sub> have a
statistically significant effect on the current value of x<sub>1</sub>, taking past
values of x<sub>1</sub> into account as regressors. We reject the null hypothesis
that x<sub>2</sub> does not Granger cause x<sub>1</sub> if the p-values are below a desired size
of the test.

In [ ]:
df[df["Volume"]>0.1][["Volume", "BTC_price"]]

In [ ]:
df_to_test = df[["Volume", "BTC_price"]]
x = df_to_test.index
y1 = df_to_test["Volume"]
y2 = df_to_test["BTC_price"]

# Plot Line1 (Left Y Axis)
fig, ax1 = plt.subplots(1,1,figsize=(16,9), dpi= 80)
ax1.plot(x, y1, color='tab:red')

# Plot Line2 (Right Y Axis)
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.plot(x, y2, color='tab:blue')

# Decorations
# ax1 (left Y axis)
ax1.set_xlabel('Date', fontsize=20)
ax1.tick_params(axis='x', rotation=0, labelsize=12)
ax1.set_ylabel('VOLUME', color='tab:red', fontsize=20)
ax1.tick_params(axis='y', rotation=0, labelcolor='tab:red' )
ax1.grid(alpha=.4)

# ax2 (right Y axis)
ax2.set_ylabel("BTC", color='tab:blue', fontsize=20)
ax2.tick_params(axis='y', labelcolor='tab:blue')
plt.show()

We will use <b>granger causalitytests</b> for Granger Causality Test from "statsmodels" library

Now let's define custom function which will do Granger Causality Test and return result as <b>pd.DataFrame</b>

In [ ]:
def grangers_causation_matrix(data, maxlag, variables, test='ssr_chi2test', verbose=False):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

In [ ]:
grangers_causation_matrix(df_to_test, 1, variables=df_to_test.columns)

How to interpret the p-values?

Assuming a significance level of 0.05, if the p-value is lesser than 0.05, then we do NOT reject the null hypothesis that X does NOT granger cause Y.

So, in the above table, the p-value for Avg_price_x and BTC_price_y is 0.02. So we accept the null hypothesis and conclude that (Avg_price) does not granger causes (BTC_price).

That means, Avg_price will not be helpful in predicting the BTC_price.

The p-value for Volume and BTC_price_x is 0. 

Since the p-value isn’t less than 0.05, we can’t reject the null hypothesis. That is, "Avg_price_x" isn’t predictive of "BTC_price_y".

Let's calculate Granger Causality Test for all available pairs

In [ ]:
currs = ["Volume","BTC_price","BNB_price"]
cols = [f"{curr}_x" for curr in currs]
idxs = [f"{curr}_y" for curr in currs]

gc_df = pd.DataFrame(columns=cols, index=idxs)

for (curr1, curr2) in itertools.combinations(currs, 2):
    df_to_test_2 = df[[curr1,curr2]]
    res_df = grangers_causation_matrix(df_to_test_2, 1, variables=df_to_test_2.columns)
    p1 = res_df[f"{curr1}_x"][f"{curr2}_y"]
    p2 = res_df[f"{curr2}_x"][f"{curr1}_y"]
    gc_df.loc[f"{curr1}_y", f"{curr2}_x"] = p1
    gc_df.loc[f"{curr2}_y", f"{curr1}_x"] = p2
    
np.fill_diagonal(gc_df.values, "—")
    
gc_df

Above we can see a matrix that consists of all the p-values. How to interpret these values? We take any value, its column will be responsible for the dependent value (X), and the row - for the independent value (Y). If that value < 0.05 then that means X granger-causes Y

<p>We now have a better idea of what our data looks like and which variables are more related to our main currency <b>MATIC</b></p>

Correlation between Matic Volume and currencies :

<ul>
    <li>BNB ~ 0.06</li>
</ul>
<ul>
    <li>BNB ~ -0.20</li>
</ul>

Best correlation fields:

<ul>
    <li>Rec_count ~ 0.69</li>
</ul>
<ul>
    <li>ATR ~ 0.55</li>
</ul>

Lets create new dataset with other currencies and best fields


In [ ]:
df[["Volume","Rec_count","BTC_price", "ATR", "BNB_price"]].to_csv("DataSet_final.csv")

# **Thank you for completing this lab!**

## Author

<a href="https://author.skills.network/instructors/ostap_liashenyk" target="_blank" >Ostap Liashenyk</a>

<a href="https://author.skills.network/instructors/yaroslav_vyklyuk_2?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX0QGDEN2306-2023-01-01">Prof. Yaroslav Vyklyuk, DrSc, PhD</a>

<a href="https://author.skills.network/instructors/mariya_fleychuk?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX0QGDEN2306-2023-01-01">Prof. Mariya Fleychuk, DrSc, PhD</a>




## Change Log

| Date (YYYY-MM-DD) | Version | Changed By      | Change Description                                         |
| ----------------- | ------- | ----------------| ---------------------------------------------------------- |
|     2023-04-01    |   1.0   | Ostap Liashenyk | Creation of the lab                                        |

<hr>

## <h3 align="center"> © IBM Corporation 2023. All rights reserved. </h3>